In [2]:
import pandas as pd
import numpy as np


In [3]:
bt2023 = pd.read_csv(r'D:\AlgoT\data\1min\backtest2023-adaptiveSL-1min.csv') 
bt2024 = pd.read_csv(r"D:\AlgoT\data\1min\backtest2024-adaptiveSL-1min.csv")
bt2022 = pd.read_csv(r"D:\AlgoT\data\1min\backtest2022-adaptiveSL-1min(New).csv")
bt2021 = pd.read_csv(r"D:\AlgoT\data\1min\backtest2021-adaptiveSL-1min.csv")
bt2020 = pd.read_csv(r"D:\AlgoT\data\1min\backtest2020-adaptiveSL-1min.csv")
bt2019 = pd.read_csv(r"D:\AlgoT\data\1min\backtest2019-adaptiveSL-1min.csv")

In [4]:
btall = [bt2019, bt2020, bt2021, bt2022, bt2023, bt2024]
data = pd.concat(btall, ignore_index=True)
data

,Unnamed: 0,Ticker,Datetime,Open,High,Low,Close,ama,rsi,atr,...,legPriceFinal3,legPriceFinal4,m2m1,m2m2,m2m3,m2m4,totalPL,cumReturns,balance,stoploss
0,0,NIFTY,2019-01-01 09:15:00+05:30,10881.70,10881.70,10881.70,10881.70,10871.122710,51.426764,3.117619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,NaN
1,1,NIFTY,2019-01-01 09:16:00+05:30,10884.10,10885.30,10872.30,10874.55,10871.125790,50.424837,3.165476,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100000.0,NaN
2,2,NIFTY,2019-01-01 09:17:00+05:30,10874.10,10879.15,10874.10,10874.90,10871.129000,50.472298,3.173571,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100000.0,NaN
3,3,NIFTY,2019-01-01 09:18:00+05:30,10873.85,10876.65,10873.85,10876.65,10871.132410,50.709366,3.173095,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100000.0,NaN
4,4,NIFTY,2019-01-01 09:19:00+05:30,10876.70,10876.70,10861.65,10863.65,10871.135430,48.960140,3.229286,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497884,34864,NaN,2024-05-17 15:25:00+05:30,22472.15,22473.15,22468.60,22469.95,22384.555014,59.023836,26.021429,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
497885,34865,NaN,2024-05-17 15:26:00+05:30,22468.80,22470.65,22464.40,22469.95,22384.555014,59.023836,26.021429,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
497886,34866,NaN,2024-05-17 15:27:00+05:30,22469.90,22482.40,22469.90,22472.75,22384.555014,59.023836,26.021429,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
497887,34867,NaN,2024-05-17 15:28:00+05:30,22472.90,22475.00,22461.20,22465.30,22384.555014,59.023836,26.021429,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [5]:

data['Datetime'] = pd.to_datetime(data['Datetime'])

def calculate_brokerage_from_positions(data):
    # Initialize variables
    total_brokerage = 0
    daily_brokerage = {}
    daily_signals = {}
    active_orders = {'call_short': 0, 'call_buy': 0, 'put_short': 0, 'put_buy': 0}
    last_signal = None
    brokerage_per_lot = 10
    current_day = None

    # Process each row of data
    for index, row in data.iterrows():
        timestamp = row['Datetime']
        signal = row['position']
        
        # Check for a new day
        if current_day != timestamp.date():
            if current_day is not None:
                daily_brokerage[current_day] = daily_signals[current_day]['brokerage']
            current_day = timestamp.date()
            daily_signals[current_day] = {'signals': [], 'brokerage': 0}
            last_signal = None  # Reset last_signal at the start of a new day

        # Only process if the signal has changed
        if signal != last_signal:
            last_signal = signal
            new_orders = 0

            if signal == 'beginx':
                active_orders = {'call_short': 1, 'call_buy': 1, 'put_short': 1, 'put_buy': 1}
                new_orders = 4
            elif signal == 'buy':
                if active_orders['call_short'] > 0:
                    active_orders['call_short'] -= 1
                active_orders['call_buy'] += 1
                active_orders['put_short'] += 1
                active_orders['put_buy'] += 1
                new_orders = 4  # 1 lot exited, 3 lots added
            elif signal == 'sell':
                if active_orders['put_short'] > 0:
                    active_orders['put_short'] -= 1
                active_orders['call_short'] += 1
                active_orders['call_buy'] += 1
                active_orders['put_buy'] += 1
                new_orders = 4  # 1 lot exited, 3 lots added
            elif signal == 'squareoff':
                new_orders = sum(active_orders.values())
                active_orders = {'call_short': 0, 'call_buy': 0, 'put_short': 0, 'put_buy': 0}

            # Update the daily brokerage and signal log
            daily_signals[current_day]['brokerage'] += new_orders * brokerage_per_lot
            total_brokerage += new_orders * brokerage_per_lot
        
        # Always record signals even if they haven't changed
        daily_signals[current_day]['signals'].append(signal)

    # Include the last day in the data
    if current_day is not None:
        daily_brokerage[current_day] = daily_signals[current_day]['brokerage']

    return daily_brokerage, total_brokerage, daily_signals

# Execute the function with the data
daily_brokerage, total_brokerage, daily_signals = calculate_brokerage_from_positions(data)

# Create and display summary of daily brokerage
df_daily_brokerage = pd.DataFrame([
    {'Date': date, 'Daily Brokerage': info['brokerage'], 'Signals': ', '.join(set(info['signals']))}
    for date, info in daily_signals.items()
])
df_daily_brokerage.sort_values(by='Date', inplace=True)

print(df_daily_brokerage)
print("Total Brokerage:", total_brokerage)


            Date  Daily Brokerage                        Signals
0     2019-01-01              140  sell, beginx, hold, squareoff
1     2019-01-02              140  sell, beginx, hold, squareoff
2     2019-01-03              140  sell, beginx, hold, squareoff
3     2019-01-04               80        beginx, hold, squareoff
4     2019-01-07               80        beginx, hold, squareoff
...          ...              ...                            ...
1322  2024-05-13              140   buy, beginx, hold, squareoff
1323  2024-05-14               80        beginx, hold, squareoff
1324  2024-05-15               80        beginx, hold, squareoff
1325  2024-05-16              220  sell, beginx, hold, squareoff
1326  2024-05-17               80        beginx, hold, squareoff

[1327 rows x 3 columns]
Total Brokerage: 138900


In [10]:
df_daily_brokerage.to_csv(r"D:\AlgoT\brokrage\brokrage1min.csv")

In [7]:
df_daily_brokerage

,Date,Daily Brokerage,Signals
0,2019-01-01,140,"sell, beginx, hold, squareoff"
1,2019-01-02,140,"sell, beginx, hold, squareoff"
2,2019-01-03,140,"sell, beginx, hold, squareoff"
3,2019-01-04,80,"beginx, hold, squareoff"
4,2019-01-07,80,"beginx, hold, squareoff"
...,...,...,...
1322,2024-05-13,140,"buy, beginx, hold, squareoff"
1323,2024-05-14,80,"beginx, hold, squareoff"
1324,2024-05-15,80,"beginx, hold, squareoff"
1325,2024-05-16,220,"sell, beginx, hold, squareoff"
